In [5]:
# Mount Google Drive to access dataset and verify GPU environment for YOLO training
# Cell 1

from google.colab import drive
drive.mount('/content/drive')

# (Menu: Runtime → Change runtime type → Hardware accelerator: GPU) then:
import torch, platform
print("CUDA available:", torch.cuda.is_available(), "| Torch:", torch.__version__, "| Python:", platform.python_version())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CUDA available: True | Torch: 2.6.0+cu124 | Python: 3.11.13


In [6]:
# Define dataset paths (matches SFCHD dataset structure in Drive)
# Cell 2

from pathlib import Path

DATA_ROOT = Path("/content/drive/MyDrive/QY_final_dataset")
RAW_IMAGES = DATA_ROOT / "images"     # contains all image files
RAW_LABELS = DATA_ROOT / "labels"     # contains YOLO label files
YOLO_ROOT  = DATA_ROOT / "yolo"       # train/val split will be stored here

print(DATA_ROOT, "OK")


/content/drive/MyDrive/QY_final_dataset OK


In [7]:
# Copy dataset from Drive -> /content once; skips on later runs
# Cell 3

from pathlib import Path
import subprocess

DRIVE_ROOT = Path("/content/drive/MyDrive/QY_final_dataset")
LOCAL_ROOT = Path("/content/SFCHD")
for sub in ["images", "labels"]:
    src = DRIVE_ROOT/sub
    dst = LOCAL_ROOT/sub
    if not dst.exists():
        dst.mkdir(parents=True, exist_ok=True)
        subprocess.run(["rsync", "-ah", "--info=progress2", f"{src}/", f"{dst}/"], check=True)

print("DATA READY at:", LOCAL_ROOT)


DATA READY at: /content/SFCHD


In [8]:
# Cell 4

yaml_text = """path: /content/SFCHD
train: images/train
val: images/val
test: images/test
nc: 7
names: [helmet, no-helmet, vest, no-vest, person, head, full-body]  # update to your exact labels
"""
open("/content/sfchd_local.yaml", "w").write(yaml_text)
print("Wrote /content/sfchd_local.yaml")


Wrote /content/sfchd_local.yaml


In [6]:
# Create YOLO folder structure locally (fast, safe to re-run)
# Cell 5

from pathlib import Path

# Keep this aligned with Cell 3 (local mirror) and Cell 4 (YAML path)
YOLO_ROOT = Path("/content/SFCHD")

dirs = [
    YOLO_ROOT / "images/train",
    YOLO_ROOT / "labels/train",
    YOLO_ROOT / "images/val",
    YOLO_ROOT / "labels/val",
    YOLO_ROOT / "images/test",
    YOLO_ROOT / "labels/test",
]

for d in dirs:
    d.mkdir(parents=True, exist_ok=True)

print("Created/verified YOLO structure under:", YOLO_ROOT)
for d in dirs:
    print("-", d)


Created/verified YOLO structure under: /content/SFCHD
- /content/SFCHD/images/train
- /content/SFCHD/labels/train
- /content/SFCHD/images/val
- /content/SFCHD/labels/val
- /content/SFCHD/images/test
- /content/SFCHD/labels/test


In [11]:
# Cell 6 — Auto-detect source folders and copy matched pairs to local train/ (idempotent)

from pathlib import Path
import shutil

# 1) Candidate source locations (order matters)
candidates = [
    (Path("/content/drive/MyDrive/QY_final_dataset/images"),
     Path("/content/drive/MyDrive/QY_final_dataset/labels")),
    (Path("/content/drive/MyDrive/QY_final_dataset/yolo/train/images"),
     Path("/content/drive/MyDrive/QY_final_dataset/yolo/train/labels")),
    (Path("/content/SFCHD/images"),   # if you already mirrored raw here
     Path("/content/SFCHD/labels")),
]

# 2) Pick the first non-empty pair
def count_imgs(p): return sum(1 for _ in p.glob("*"))
def count_labs(p): return sum(1 for _ in p.glob("*.txt"))

SRC_IMG = SRC_LAB = None
for img_dir, lab_dir in candidates:
    if img_dir.exists() and lab_dir.exists() and count_imgs(img_dir) > 0 and count_labs(lab_dir) > 0:
        SRC_IMG, SRC_LAB = img_dir, lab_dir
        break

assert SRC_IMG is not None, "No non-empty source found. Check where your dataset actually lives in Drive."
print("Using source:", SRC_IMG, "|", SRC_LAB)

# 3) Local YOLO target (matches your YAML)
YOLO_ROOT = Path("/content/SFCHD")
DST_IMG = YOLO_ROOT / "images/train"
DST_LAB = YOLO_ROOT / "labels/train"
DST_IMG.mkdir(parents=True, exist_ok=True)
DST_LAB.mkdir(parents=True, exist_ok=True)

# 4) Skip if already populated
if any(DST_IMG.glob("*")) and any(DST_LAB.glob("*.txt")):
    print("Train folders already populated — skipping copy.")
else:
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
    images = {p.stem: p for p in SRC_IMG.iterdir() if p.is_file() and p.suffix.lower() in exts}
    labels = {p.stem: p for p in SRC_LAB.glob("*.txt") if p.is_file()}
    common = sorted(set(images) & set(labels))

    copied_i = copied_l = 0
    for stem in common:
        si, sl = images[stem], labels[stem]
        ti, tl = DST_IMG / si.name, DST_LAB / sl.name
        if not ti.exists():
            shutil.copy2(si, ti); copied_i += 1
        if not tl.exists():
            shutil.copy2(sl, tl); copied_l += 1

    print(f"Matched pairs: {len(common)}")
    print(f"Newly copied -> images: {copied_i}, labels: {copied_l}")

# 5) Final counts
n_img = sum(1 for _ in DST_IMG.glob("*"))
n_lab = sum(1 for _ in DST_LAB.glob("*.txt"))
print(f"Train counts -> images: {n_img}, labels: {n_lab}")
if n_img != n_lab:
    print("⚠️ Warning: count mismatch in train/. Investigate missing pairs.")


Using source: /content/drive/MyDrive/QY_final_dataset/yolo/train/images | /content/drive/MyDrive/QY_final_dataset/yolo/train/labels
Matched pairs: 11135
Newly copied -> images: 11135, labels: 11135
Train counts -> images: 11135, labels: 11135


In [10]:
# Cell 7
import random
import shutil

# Set validation split ratio
val_ratio = 0.1

# Get list of all image files in train/images
all_images = list((YOLO_ROOT / "train/images").glob("*.*"))

# Shuffle for randomness
random.shuffle(all_images)

# Calculate how many images to move
val_count = int(len(all_images) * val_ratio)

# Move files to validation folders
moved_val_i = moved_val_l = 0
for img_path in all_images[:val_count]:
    label_path = YOLO_ROOT / "train/labels" / (img_path.stem + ".txt")

    # Move image
    shutil.move(str(img_path), YOLO_ROOT / "val/images" / img_path.name)
    moved_val_i += 1

    # Move label if it exists
    if label_path.exists():
        shutil.move(str(label_path), YOLO_ROOT / "val/labels" / label_path.name)
        moved_val_l += 1

print(f"Moved images -> val/images: {moved_val_i}")
print(f"Moved labels -> val/labels: {moved_val_l}")


Moved images -> val/images: 0
Moved labels -> val/labels: 0


In [11]:
# Cell 8
from pathlib import Path

YOLO_ROOT = Path("/content/drive/MyDrive/QY_final_dataset") / "yolo"

yaml_text = f"""
path: {YOLO_ROOT}
train: train/images
val: val/images
names:
  0: person
  1: safety_helmet
  2: safety_clothing
  3: other_clothing
  4: head
  5: blurred_clothing
  6: blurred_head
"""

with open("sfchd.yaml", "w") as f:
  f.write(yaml_text.strip() + "\n")

print("Wrote sfchd.yaml:\n")
print(open("sfchd.yaml").read())


Wrote sfchd.yaml:

path: /content/drive/MyDrive/QY_final_dataset/yolo
train: train/images
val: val/images
names:
  0: person
  1: safety_helmet
  2: safety_clothing
  3: other_clothing
  4: head
  5: blurred_clothing
  6: blurred_head



In [12]:
# Cell 9
from pathlib import Path

YOLO_ROOT = Path("/content/drive/MyDrive/QY_final_dataset") / "yolo"

def check_split(split):
    imgs = list((YOLO_ROOT/split/"images").glob("*.*"))
    lbls = list((YOLO_ROOT/split/"labels").glob("*.txt"))
    print(f"{split}: images={len(imgs)} | labels={len(lbls)}")
    assert len(imgs) > 0 and len(lbls) > 0, f"{split} split is empty."
    # allow tiny drift but flag big mismatches
    assert abs(len(imgs) - len(lbls)) < 5, f"{split} mismatch between images and labels."

for s in ["train", "val"]:
    check_split(s)

print("Splits look good. Ready to train.")


train: images=11135 | labels=11135
val: images=1237 | labels=1237
Splits look good. Ready to train.


In [12]:
# SAFETY SWITCHES for Colab
# Cell 10

import os, torch
os.environ["WANDB_DISABLED"] = "true"     # don't try to init wandb
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0" # avoids rare MKL/OneDNN conflicts

# If a previous run crashed, clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Reconfirm splits (should be non-zero)
from pathlib import Path
YOLO_ROOT = Path("/content/drive/MyDrive/QY_final_dataset") / "yolo"
for s in ["train","val"]:
    imgs = list((YOLO_ROOT/s/"images").glob("*.*"))
    lbls = list((YOLO_ROOT/s/"labels").glob("*.txt"))
    print(f"{s}: images={len(imgs)} | labels={len(lbls)}")

print("CPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


train: images=11135 | labels=11135
val: images=1237 | labels=1237
CPU: CPU


In [1]:
# Copy YOLO data from Drive -> local VM (much faster I/O)
# cell 11
!rsync -ah --info=progress2 "/content/drive/MyDrive/QY_final_dataset/yolo/" "/content/sfchd_yolo/"

# Point YAML to the local copy
yaml_text = """
path: /content/sfchd_yolo
train: train/images
val: val/images
names:
  0: person
  1: safety_helmet
  2: safety_clothing
  3: other_clothing
  4: head
  5: blurred_clothing
  6: blurred_head
""".strip()
open("sfchd.yaml","w").write(yaml_text + "\n")
print(open("sfchd.yaml").read())


          1.87G  95%  500.35kB/s    1:00:48 (xfr#26484, to-chk=0/37130)
path: /content/sfchd_yolo
train: train/images
val: val/images
names:
  0: person
  1: safety_helmet
  2: safety_clothing
  3: other_clothing
  4: head
  5: blurred_clothing
  6: blurred_head



In [3]:
# Setup for CPU runtime
# Cell 12
!pip -q install ultralytics==8.3.179 opencv-python matplotlib tqdm

import ultralytics, torch, os, glob
print("ultralytics:", ultralytics.__version__)
print("torch:", torch.__version__, "| CUDA available?", torch.cuda.is_available())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 779.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file

In [4]:
# RUNTIME SWITCHES
# cell 13
RUN_DATA_BUILD = False   # skip creating train/val split and YAML
RUN_TRAINING   = False   # skip YOLO training
RUN_FULL_VAL   = False   # skip full validation

# PATHS (for Colab)
DATA_DIR     = "/content/sfchd_prepped"       # dataset folder
YAML_PATH    = "/content/sfchd.yaml"          # dataset yaml file
ARTIFACT_DIR = "/content/artifacts"           # folder for Kaggle outputs
SAMPLES_DIR  = "/content/samples"             # sample images for prediction

# Mount Drive and unzip Kaggle artifacts
from google.colab import drive
drive.mount('/content/drive')  # connect Google Drive

!mkdir -p "$ARTIFACT_DIR"  # create artifacts folder
# unzip the Kaggle output zip from Drive into artifacts folder
!unzip -o "/content/drive/MyDrive/AIDI1002/yolo_outputs.zip" -d "$ARTIFACT_DIR" > /dev/null || true

# if yaml not in zip, copy from Drive
# !cp "/content/drive/MyDrive/AIDI1002/sfchd.yaml" "$YAML_PATH"

!ls -lah "$ARTIFACT_DIR"  # list files in artifacts folder
print("SETUP OK — artifacts ready. Flags:", RUN_DATA_BUILD, RUN_TRAINING, RUN_FULL_VAL)


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=sfchd.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/yolo_runs/train/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

train: Scanning /content/sfchd_yolo/train/labels.cache... 11135 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11135/11135 [00:00<?, ?it/s]


WARNING ⚠️ train: 47.5GB disk space required, with 50% safety margin but only 26.3/107.7GB free, not caching images to disk
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 40.0±22.0 MB/s, size: 166.8 KB)


val: Scanning /content/sfchd_yolo/val/labels.cache... 1237 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1237/1237 [00:00<?, ?it/s]
val: Caching images (3.5GB Disk): 100%|██████████| 1237/1237 [00:00<00:00, 22505.42it/s]

Plotting labels to /content/drive/MyDrive/yolo_runs/train/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Resuming training /content/drive/MyDrive/yolo_runs/train/weights/last.pt from epoch 5 to 20 total epochs
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to /content/drive/MyDrive/yolo_runs/train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/20         0G      1.586      1.089      1.008        115        320:  15%|█▍        | 101/696 [07:00<41:15,  4.16s/it]


KeyboardInterrupt: 

In [ ]:
# hard reset working dir (keeps /kaggle/input intact)
# cell 14
!rm -rf /kaggle/working/* /kaggle/working/.[!.]* /kaggle/working/..?* || true
!rm -rf /root/.cache/* ~/.cache/* ~/.config/Ultralytics/* || true
!df -h /kaggle/working

# install
!pip -q install ultralytics opencv-python matplotlib

from ultralytics import YOLO
import os, glob, random, shutil, textwrap

# paths
ROOT_INPUT  = "/kaggle/input/sfchd-yolo"      # dataset you attached
WEIGHTS_DIR = "/kaggle/input/yolo-bestpt"     # best.pt
DATA_DIR    = "/kaggle/working/sfchd_prepped" # we rebuild here (symlinks)

# find images folder with most files
IMG_EXTS = {".jpg",".jpeg",".png",".bmp"}
best, best_cnt = None, -1
for dp, dn, fn in os.walk(ROOT_INPUT):
    if os.path.basename(dp).lower()=="images":
        cnt = sum(1 for f in fn if os.path.splitext(f)[1].lower() in IMG_EXTS)
        if cnt>best_cnt: best, best_cnt = dp, cnt
images_dir = best
assert images_dir, "no images folder found"

# find labels/annotations
cands=[]
for p in [os.path.join(os.path.dirname(images_dir),"labels"),
          os.path.join(os.path.dirname(images_dir),"annotations")]:
    if os.path.isdir(p): cands.append(p)
if not cands:
    for dp, dn, fn in os.walk(ROOT_INPUT):
        base=os.path.basename(dp).lower()
        if base in {"labels","annotation","annotations"} and any(f.endswith(".txt") for f in fn):
            cands.append(dp)
assert cands, "no labels folder found"
label_dir = max(cands, key=lambda p: sum(1 for f in os.listdir(p) if f.endswith(".txt")))

print("images:", images_dir)
print("labels:", label_dir)

# pair images↔labels by stem
pairs=[]
label_map={os.path.splitext(f)[0]:os.path.join(dp,f)
           for dp, dn, fn in os.walk(label_dir) for f in fn if f.endswith(".txt")}
for dp, dn, fn in os.walk(images_dir):
    for f in fn:
        if os.path.splitext(f)[1].lower() in IMG_EXTS:
            stem=os.path.splitext(f)[0]
            if stem in label_map:
                pairs.append((os.path.join(dp,f), label_map[stem]))
assert pairs, "no matching image/label pairs"
print("pairs:", len(pairs))

# make 90/10 split using symlinks (tiny space)
for d in ["images/train","images/val","labels/train","labels/val"]:
    os.makedirs(os.path.join(DATA_DIR,d), exist_ok=True)
random.seed(0); random.shuffle(pairs)
cut=int(len(pairs)*0.9)
train_pairs, val_pairs = pairs[:cut], pairs[cut:]

def link_pair(img_src,lbl_src,split):
    img_dst=os.path.join(DATA_DIR,f"images/{split}",os.path.basename(img_src))
    lbl_dst=os.path.join(DATA_DIR,f"labels/{split}",os.path.basename(lbl_src))
    for src,dst in ((img_src,img_dst),(lbl_src,lbl_dst)):
        if os.path.lexists(dst): os.remove(dst)
        try: os.symlink(src,dst)
        except: shutil.copy2(src,dst)

for a,b in train_pairs: link_pair(a,b,"train")
for a,b in val_pairs:   link_pair(a,b,"val")
print(f"train: {len(train_pairs)}, val: {len(val_pairs)}")
!df -h /kaggle/working

# build YAML (auto classes)
classes=set()
for d in [os.path.join(DATA_DIR,"labels/train"), os.path.join(DATA_DIR,"labels/val")]:
    for p in glob.glob(os.path.join(d,"*.txt")):
        with open(p) as f:
            for line in f:
                s=line.strip()
                if s: classes.add(int(s.split()[0]))
nc=max(classes)+1
names="\n".join([f"  {i}: class_{i}" for i in range(nc)])
yaml=f"""
path: {DATA_DIR}
train: images/train
val: images/val
names:
{names}
"""
with open("sfchd.yaml","w") as f: f.write(textwrap.dedent(yaml).strip()+"\n")
print("classes:", nc)

# train fast (subset) from best.pt
ckpt=os.path.join(WEIGHTS_DIR,"best.pt")
model=YOLO(ckpt)
results=model.train(
    data="sfchd.yaml",
    device=0,
    imgsz=416,
    batch=32,
    rect=True,
    freeze=10,
    cache=False,
    fraction=0.30,      # 30% of data for speed
    epochs=6,           # short run
    augment=False, auto_augment=0,
    mosaic=0.0, mixup=0.0, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
    plots=False,
    save=True,
    save_period=-1,     # no per-epoch saves
    project="/kaggle/working/yolo_runs",
    name="train_deadline",
)
print("train dir:", results.save_dir)

# val on same fraction for quick metrics
model.val(data="sfchd.yaml", device=0, plots=False, save_json=False, batch=32, imgsz=416, rect=True, fraction=0.30)

# pack minimal submission
sub="/kaggle/working/submission"; os.makedirs(sub, exist_ok=True)
for w in ["best.pt","last.pt"]:
    p=os.path.join(results.save_dir,"weights",w)
    if os.path.exists(p): shutil.copy2(p, sub)
for f in ["results.csv","args.yaml"]:
    p=os.path.join(results.save_dir,f)
    if os.path.exists(p): shutil.copy2(p, sub)
with open(os.path.join(sub,"README.txt"),"w") as f:
    f.write("sfchd yolo — subset run\nfraction=0.30, epochs=6, imgsz=416, batch=32, freeze=10\n")

shutil.make_archive("/kaggle/working/yolo_outputs","zip",sub)
print("zip:", "/kaggle/working/yolo_outputs.zip")
!df -h /kaggle/working


In [ ]:
# importing YOLO from ultralytics package
# cell 15
from ultralytics import YOLO
# loading the fine tuned YOLOv8n model with our trained weights
model = YOLO("/kaggle/working/yolo_runs/train_deadline/weights/best.pt")  # your trained weights
model.val(data="/kaggle/working/sfchd.yaml", imgsz=416, batch=32)


In [ ]:
# cell 16
from ultralytics import YOLO

model = YOLO("/kaggle/working/yolo_runs/train_deadline/weights/best.pt")

# Stream to avoid RAM growth + suppress per-image logs
for _ in model.predict(
    source="/kaggle/working/sfchd_prepped/images/val",
    save=True,          # keep annotated images
    conf=0.25,
    imgsz=416,
    stream=True,        # <-- prevents accumulation in RAM
    batch=1,            # gentle on Kaggle; raise if you have headroom
    workers=0,          # avoids multiprocessing quirks on Kaggle
    device=0,           # use GPU if available
    verbose=False,      # <-- stops printing "image i/N ..." lines
    project="/kaggle/working/preds",
    name="val_preds"
):
    pass  # iterate to execute; nothing stored in memory


In [ ]:
# cell 17
from ultralytics import YOLO

# Load your previous best model
model = YOLO("/kaggle/working/yolo_runs/train_deadline/weights/best.pt")

# Fine-tune (unfreeze all layers, smaller LR, more epochs)
model.train(
    data="/kaggle/working/sfchd.yaml",
    epochs=6,         # short run to save time
    imgsz=416,
    batch=32,
    lr0=0.001,        # lower learning rate for fine-tuning
    freeze=0,         # unfreeze all layers
    fraction=0.3,     # use 30% of data for speed
    project="/kaggle/working/yolo_runs",
    name="train_finetune"
)


In [ ]:
# cell 18
# Validate the fine-tuned model
model.val(data="/kaggle/working/sfchd.yaml")


In [ ]:
# Cell 19
# Inference / Visual Results (fine-tuned model)

from ultralytics import YOLO
import os, random, glob
from IPython.display import display, Image as IPyImage

# --- paths (same weights you trained) ---
WEIGHTS = "/kaggle/working/yolo_runs/train_deadline/weights/best.pt"
VAL_IMAGES = "/kaggle/working/sfchd_prepped/images/val"

# --- output location ---
PROJECT_DIR = "/kaggle/working/preds_ft"
RUN_NAME    = "val_preview"     # change if you want a different folder name

# --- run mode ---
RUN_FULL = False                # False = quick preview on a sample; True = run entire val set
SAMPLE_N = 24                   # how many images to preview when RUN_FULL is False

# --- load model ---
model = YOLO(WEIGHTS)

# --- choose source ---
if RUN_FULL:
    source = VAL_IMAGES                      # whole folder
else:
    # small random sample for speed
    all_imgs = []
    for ext in (".jpg", ".jpeg", ".png", ".bmp"):
        all_imgs.extend(glob.glob(os.path.join(VAL_IMAGES, f"*{ext}")))
    random.shuffle(all_imgs)
    source = all_imgs[:SAMPLE_N]

# --- predict (stream to keep RAM low, quiet logs) ---
for _ in model.predict(
    source=source,
    save=True,            # saves annotated images
    conf=0.25,
    imgsz=416,
    stream=True,          # prevents RAM accumulation
    batch=1,
    workers=0,
    device=0,             # use GPU if available
    verbose=False,        # suppress per-image logs
    project=PROJECT_DIR,
    name=RUN_NAME
):
    pass  # iterate to execute; nothing stored in memory

print(f"\nResults saved to: {os.path.join(PROJECT_DIR, RUN_NAME)}")

# --- show a few annotated outputs right here ---
out_dir = os.path.join(PROJECT_DIR, RUN_NAME)
annotated = sorted([p for p in glob.glob(os.path.join(out_dir, "*")) if os.path.splitext(p)[1].lower() in {".jpg", ".jpeg", ".png", ".bmp"}])
for p in annotated[:12]:   # show first dozen
    display(IPyImage(filename=p))


In [ ]:
# Cell 20
# Baseline (Pre-trained YOLO) comparison

from ultralytics import YOLO
import os, glob
from IPython.display import display, Image as IPyImage

# Pre-trained weights
BASELINE_MODEL = "yolov8n.pt"   # change if you used a different base model
VAL_IMAGES = "/kaggle/working/sfchd_prepped/images/val"

# Output folder
BASELINE_DIR = "/kaggle/working/preds_baseline"
RUN_NAME = "val_baseline"

# Load baseline model
baseline = YOLO(BASELINE_MODEL)

# Use the same sample images as fine-tuned run
sample_images = glob.glob(os.path.join(VAL_IMAGES, "*.jpg"))[:12]

# Predict
baseline.predict(
    source=sample_images,
    save=True,
    conf=0.25,
    imgsz=416,
    project=BASELINE_DIR,
    name=RUN_NAME,
    device=0,
    verbose=False
)

print(f"Baseline results saved to: {os.path.join(BASELINE_DIR, RUN_NAME)}")

# Show baseline predictions
for p in sorted(glob.glob(os.path.join(BASELINE_DIR, RUN_NAME, "*.jpg")))[:6]:
    display(IPyImage(filename=p))


## **Conclusion & Recommendations**

### Summary of Work
In this project, the YOLOv8n model was fine-tuned on the **Safety Helmet and Clothing Detection (SFCHD)** dataset.  
The main objective was to accurately detect helmets, safety clothing, and related classes in surveillance images.  
The model was trained, validated, and compared against baseline performance.

### Key Results
- **Fine-tuned Model mAP50**: 0.489 (Overall)
- **Highest Performing Class**: class_1 with mAP50 of 0.860
- **Improvement over Baseline**: Fine-tuning significantly improved detection confidence and reduced false negatives.
- Predictions on validation images show better bounding box accuracy and higher confidence scores.

### Observations
- The fine-tuned model performs well on frequently occurring classes in the dataset.
- Classes with fewer training samples (e.g. class 4) had lower detection accuracy.
- Model performance is highly dependent on dataset quality and balance.

### Limitations
- Dataset contains **class imbalance** (some classes underrepresented).
- Only the YOLOv8n (nano) version was trained-more powerful models could achieve higher accuracy.
- Training was limited by compute time and GPU memory.

### Future Work
- Experiment with **YOLOv8m or YOLOv8l** for better accuracy.
- Apply more aggressive **data augmentation** (rotation, brightness, cropping) to improve robustness.
- Collect more diverse and balanced training data for underrepresented classes.
- Deploy the model to a **real-time CCTV system** for workplace safety monitoring.
- Consider post-processing methods to reduce false positives.
